In [1]:
from QWorker import IPythonDisplay


In [ ]:
#__worker_class__

class Worker(object):

    def columns(self):
        return ["NJets"]

    def process(self, events, emit):
        emit("NJets", events.NJets)
        

In [ ]:
from QWorker import DistributedStripedSession as Session
from QWorker import IPythonDisplay
from datasets import Datasets

from histbook import Hist, beside, groupby, below
from histbook import bin as hbin

registry_url = "http://ifdb01.fnal.gov:9867"
data_server_url = "http://dbweb7.fnal.gov:9091/striped/app"
    
session = Session(data_server_url, registry_url)

h_by_dataset = Hist(hbin("NJets", 10, 0, 10), groupby("dataset"))
h_combined = Hist(hbin("NJets", 20, 0, 20))
#h_all = Hist(hbin("NJets", 20, 0, 20))

h_by_dataset_display = IPythonDisplay(
    h_by_dataset
        .stack("dataset")
        .area("NJets", width=400)
)
h_combined_display = IPythonDisplay(
    h_combined
        .step("NJets", width=400)
)

display = IPythonDisplay(
    beside(
        h_by_dataset
            .stack("dataset")
            .area("NJets", width=100),
        h_combined
            .line("NJets", width=100, yscale={"type":"log"})
    )
)

def callback(typ, nevents, data):
    display.update()

for dataset_name in Datasets[:15]:
    job = session.createJob(dataset_name, user_callback=callback)
    job.addHistogram(h_by_dataset, ["NJets", "dataset"], constants={"dataset":dataset_name})
    job.addHistogram(h_combined, "NJets")
    job.run()
    runtime = job.TFinish - job.TStart
    nevents = job.EventsProcessed
    nworkers = len(job.WorkerAddresses)
    display.update()

